In [1]:
import pandas as pd

df = pd.read_csv('dataFrame.csv') ##check query.sql for the data source

In [2]:
#Truncates timestamp to hour
df['FIRST_DEPOSIT_DATE_hour'] = df.FIRST_DEPOSIT_DATE.astype('<M8[h]')

#  Counts all addresses that made their first sturdy deposit in the same hour (i.e. between 6PM and 7PM on 2022/10/01)
## And were funded by the same address
suspects = df.groupby('FIRST_DEPOSIT_DATE_hour')['FROM_ADDRESS'].value_counts().reset_index(allow_duplicates = True)

##rename columns for merging with main df
suspects.columns = ['FIRST_DEPOSIT_DATE_hour','FROM_ADDRESS','FROM_ADDRESS_count']

#Merge addresses funded by suspects that made their first deposits on the suspect deposit hours
df = df.merge(suspects, how='inner', on=['FIRST_DEPOSIT_DATE_hour','FROM_ADDRESS'])

In [3]:
##output
df[df['FROM_ADDRESS_count']>1] 

,USER,TO_ADDRESS,FIRST_SEEN_ON_NETWORK,FIRST_DEPOSIT_DATE,NO_DEPOSITS,FROM_ADDRESS,TX_RANK,FIRST_DEPOSIT_DATE_hour,FROM_ADDRESS_count
4,0x94649845604d943cf105220045054f8bb3e3ce21,0x94649845604d943cf105220045054f8bb3e3ce21,2022-06-26 19:59:39.000,2022-12-06 06:38:59.000,1,0xf60c2ea62edbfe808163751dd0d8693dcb30019c,1,2022-12-06 06:00:00,9
5,0x5be4797e82a4826664c8431af4be44b82d212299,0x5be4797e82a4826664c8431af4be44b82d212299,2022-06-25 19:20:35.000,2022-12-06 06:40:59.000,1,0xf60c2ea62edbfe808163751dd0d8693dcb30019c,1,2022-12-06 06:00:00,9
6,0x1429bb62c3fc8cbfcbddf88e49e04bd93be00670,0x1429bb62c3fc8cbfcbddf88e49e04bd93be00670,2022-06-26 20:01:11.000,2022-12-06 06:36:59.000,1,0xf60c2ea62edbfe808163751dd0d8693dcb30019c,1,2022-12-06 06:00:00,9
7,0x393d635b39f2351aefcf84cf5f8c755a038cb427,0x393d635b39f2351aefcf84cf5f8c755a038cb427,2022-06-26 20:00:05.000,2022-12-06 06:37:59.000,1,0xf60c2ea62edbfe808163751dd0d8693dcb30019c,1,2022-12-06 06:00:00,9
8,0x48bc6c71f73f171a5deee36c59f8c51542504060,0x48bc6c71f73f171a5deee36c59f8c51542504060,2022-06-26 20:02:49.000,2022-12-06 06:35:47.000,1,0xf60c2ea62edbfe808163751dd0d8693dcb30019c,1,2022-12-06 06:00:00,9
...,...,...,...,...,...,...,...,...,...
554,0x8286acba2bc29a288ed0f7310c32d6b800a70907,0x8286acba2bc29a288ed0f7310c32d6b800a70907,2021-11-15 14:00:20.000,2022-11-27 07:29:23.000,1,0xc098b2a3aa256d2140208c3de6543aaef5cd3a94,1,2022-11-27 07:00:00,4
840,0xda0a3c890603ae8248df00bdfecc49a94ce2dac2,0xda0a3c890603ae8248df00bdfecc49a94ce2dac2,2021-11-30 06:01:32.000,2022-10-21 04:08:59.000,1,0x46340b20830761efd32832a74d7169b29feb9758,1,2022-10-21 04:00:00,2
841,0x7d733f761559743f244766e402d0ea014c18d9fd,0x7d733f761559743f244766e402d0ea014c18d9fd,2022-10-16 01:34:35.000,2022-10-21 04:36:59.000,2,0x46340b20830761efd32832a74d7169b29feb9758,1,2022-10-21 04:00:00,2
851,0x85c49aca0abb871e5c876e6e5d5395bd88fa3f20,0x85c49aca0abb871e5c876e6e5d5395bd88fa3f20,2022-01-22 06:01:24.000,2022-12-01 12:00:59.000,1,0x21a31ee1afc51d94c2efccaa2092ad1028285549,1,2022-12-01 12:00:00,2
